In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

## Data loading and cleasing

In [2]:
employment = pd.read_csv('new_employment.csv')
income = pd.read_csv('new_income.csv')
pop =  pd.read_csv('new_population.csv')
fac = pd.read_csv('new_child_care.csv')
loc = pd.read_csv('new_potential_loc.csv')

# population is uniform within each slot
pop['10-12'] = pop['10-14'] * 3 / 5

# identifying high-demand zipcodes
zip = income.merge(employment, how="left", left_on="zip_code", right_on="zip_code")
zip['is_high_demand'] = (zip['employment rate'] >= 0.6) | (zip['average income'] < 60000)

# arranging cols to make a zip dataframe containing information on the area
slots = fac.groupby("zip_code").sum()[['infant_capacity', 'toddler_capacity', 'preschool_capacity', 'school_age_capacity', 'children_capacity', 'total_capacity']]
zip = zip.merge(slots, how="left", left_on="zip_code", right_on="zip_code")
zip = zip.merge(pop[['zip_code', '-5', '5-9', '10-12']], left_on="zip_code", right_on="zip_code")

#identifying deserts
zip['is_desert'] = (zip['is_high_demand'] & (zip['total_capacity'] <= 1/2 * (zip['-5'] + zip['5-9'] + zip['10-12']))) | \
                   (~zip['is_high_demand'] & (zip['total_capacity'] <= 1/3 * (zip['-5'] + zip['5-9'] + zip['10-12'])))


#print(pop)
fac = fac.merge(pop,how="left",left_on="zip_code", right_on="zip_code")
#test = fac.groupby('facility_id').count()
#print(test[test["zip_code"]>1])



# cleaning data
fac = fac[fac['total_capacity'] > 0]
zip['demand_child'] = zip['-5'] + zip['5-9'] + zip['10-12']
zip['under5_capacity'] = zip['infant_capacity'] + zip['toddler_capacity'] + zip['preschool_capacity']
fac['under5_capacity'] = fac['infant_capacity'] + fac['toddler_capacity'] + fac['preschool_capacity']
zip = zip[zip['demand_child'] > 0]

new_fac_cost = {
    'size': ['small', 'medium', 'large'],
    'total_slots': [100, 200, 400],
    'under5_slots': [50, 100, 200],
    'cost': [65000, 95000, 115000]
}
new_fac_cost = pd.DataFrame(new_fac_cost)
new_fac_cost


fac = fac.reset_index(drop=True)
zip = zip.reset_index(drop=True)

zip.head(20)
fac.head(10)



# identifying key elements
zipcode = zip['zip_code'].tolist()
new_fac_size = ['small', 'medium', 'large']
exist_fac = fac['facility_id'].unique()
age = ['under5', 'over5']


## Solving for question 1

In [12]:
m = Model("child_solution")

In [14]:
# Decision Variables：new facilities & slot expansion
x = {}
for i in zipcode:
    for j in new_fac_size:
        x[i, j] = m.addVar(vtype = GRB.INTEGER, name = f"x_{i}_{j}")

y = {}
for f in exist_fac:
    for a in age:
        y[f, a] = m.addVar(vtype = GRB.INTEGER, name = f"y_{f}_{a}")


In [16]:
# Objective Function：Min the cost of...
new_fac_cost_expr = quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['cost'].values[0] for i in zipcode for j in new_fac_size)
expand_cost_expr = sum(((20000 + (y[f, 'under5'] + y[f, 'over5']) * 200) * (y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] + 100 * y[f, 'under5']) for f in exist_fac)

m.setObjective(new_fac_cost_expr + expand_cost_expr, GRB.MINIMIZE)


In [17]:
# Constraints:
# (1) no desert in both high demand and normal demand region
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[fac['zip_code'] == i]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == i]['is_high_demand'].values[0], f"no_desert_high_demand_{i}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[fac['zip_code'] == i]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{i}")


In [18]:
# (2) under5 take up a higher coverage percantage
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, 'under5'] for f in fac[fac['zip_code'] == i]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == i]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == i]['-5'].values[0], f"under5_demand_{i}")


In [19]:
# (3) expansion maximum scale
for f in exist_fac:
    m.addConstr(quicksum(y[f, a] for a in age) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.20, f"expansion_restriction_{f}")


In [21]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17815 rows, 32570 columns and 112450 nonzeros
Model fingerprint: 0xfa870b48
Model has 44265 quadratic objective terms
Variable types: 0 continuous, 32570 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+02]
  Objective range  [2e+01, 1e+05]
  QObjective range [4e-01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-04, 8e+03]
Found heuristic solution: objective 5.610000e+17
Presolve removed 16523 rows and 22896 columns (presolve time = 7s) ...
Presolve removed 16523 rows and 22896 columns
Presolve time: 6.95s
Presolved: 1292 rows, 9674 columns, 12651 nonzeros
Presolved model has 10079 quadratic objective terms
Found heuristic solution: objective 9.573037e+08
Variable types: 0 continuous, 9674 integer (552 binary)
Fo

In [20]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found. Results:")
    # 新设施的建设情况
    for i in zipcode:
        for j in new_fac_size:
            if x[i, j].x > 0:
                print(f"Region {i}: Build {x[i, j].x} {j} size new facilities.")
    
    # 现有设施kuojian情况
    for f in exist_fac:
        for a in age:
            if y[f, a].x > 0:
                print(f"Facility {f}: Expand capacity for age group {a} by {y[f, a].x}.")
elif m.status == GRB.INFEASIBLE:
    print("The model is infeasible. Check constraints or data.")
elif m.status == GRB.UNBOUNDED:
    print("The model is unbounded. Consider revising the model.")
else:
    print(f"Optimization was stopped with status {m.status}")


Optimal solution found. Results:
Region 10001: Build 2.0 large size new facilities.
Region 10002: Build 3.0 large size new facilities.
Region 10003: Build 3.0 large size new facilities.
Region 10004: Build 1.0 small size new facilities.
Region 10004: Build 1.0 large size new facilities.
Region 10005: Build 2.0 large size new facilities.
Region 10006: Build 1.0 small size new facilities.
Region 10007: Build 2.0 large size new facilities.
Region 10009: Build 5.0 large size new facilities.
Region 10010: Build 5.0 large size new facilities.
Region 10011: Build 2.0 large size new facilities.
Region 10012: Build 1.0 small size new facilities.
Region 10012: Build 2.0 large size new facilities.
Region 10013: Build 4.0 large size new facilities.
Region 10014: Build 1.0 large size new facilities.
Region 10016: Build 1.0 small size new facilities.
Region 10016: Build 5.0 large size new facilities.
Region 10017: Build 1.0 large size new facilities.
Region 10019: Build 1.0 small size new facilities

## Solving for question 2: Realistic Capacity Expansion and Distance

In [19]:
# defining usful functions in question 2
from haversine import haversine, Unit

potential_loc = pd.read_csv("new_potential_loc.csv")
potential_loc['location_string'] = potential_loc['latitude'].astype(str)+","+potential_loc['longitude'].astype(str)
potential_loc.head(10)

def loc_distance(lat1, lon1, lat2,lon2):
    distance = haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)
    return distance
#print(cost_func(300,13))
#print(calc_dist([70,40],[68,38]))


### setting variables

In [33]:

import gurobipy as gp
from gurobipy import GRB
m = Model("Realistic Capacity Expansion and Distance")


x = {}
loc_list = []
loc_str_list = []

for row in range(len(potential_loc)):
    #print()
    loc = potential_loc.loc[row,['latitude','longitude']].tolist()
    loc_list.append(loc)
    lat = loc[0]
    lon = loc[1]
    loc_string = str(lat)+","+str(lon)
    loc_str_list.append(loc_string)
    #print(loc)
    for size in new_fac_size:
        x[loc_string, size] = m.addVar(vtype = GRB.BINARY, name = f"x_{loc_string}_{size}")
        #print(loc_string)
        #print(zipzip,loc,size)


# find all pairs of locations with distance <= 0.06 and constrain on their spontaneous establishments later
pairs = []
for code in potential_loc['zip_code'].unique():
    print(code)
    workbook = potential_loc[potential_loc['zip_code']==code]
    for i in workbook['location_string'].unique().tolist():
        for j in workbook['location_string'].unique().tolist():
            if loc_str_list.index(i) < loc_str_list.index(j):
                #print(i,j)
                if loc_distance(float(i.split(",")[0]),float(i.split(",")[1]),float(j.split(",")[0]),float(j.split(",")[1])) <= 0.06:
                    #print(f"found {i}, {j}")
                    pairs.append((i,j))
                    #pairs.append((code,i,j))

print(pairs)

# number of expansion slots for 0-5y to existing factories 
y = {}
for f in exist_fac:
    y[f] = m.addVar(vtype = GRB.INTEGER, name = f'y_{f}')

# number of expansion slots to existing factories for each step, 
steps = ['step1', 'step2', 'step3']
z = {}
for f in exist_fac:
    for s in steps:
        z[f, s] = m.addVar(vtype = GRB.INTEGER, name = f'step_{f}_{s}')


# tf1 
#tf1 = {}
#tf2 = {}
#tf3 = {}
#for f in exist_fac:
#    tf1[f] = m.addVar(vtype = GRB.INTEGER, name = f'total1_{f}')
#    tf2[f] = m.addVar(vtype = GRB.INTEGER, name = f'total2_{f}')
#    tf3[f] = m.addVar(vtype = GRB.INTEGER, name = f'total3_{f}')


# the amount of total expansion from each steps (<10%, 10%~15%, >15%)
#c1 = {}
#c2 = {}
#c3 = {}
#for f in exist_fac:
#    c1[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c1_{f}')
#    c2[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c2_{f}')
#    c3[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c3_{f}')



        

10001
10002
10003
10004
10005
10006
10007
10009
10010
10011
10012
10013
10014
10016
10017
10019
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10044
10065
10069
10075
10128
10280
10282
10301
10302
10303
10304
10305
10306
10307
10308
10309
10310
10312
10314
10451
10452
10453
10454
10455
10456
10457
10458
10459
10460
10461
10462
10463
10464
10465
10466
10467
10468
10469
10470
10471
10472
10473
10474
10475
10502
10504
10506
10507
10509
10510
10511
10512
10514
10516
10520
10522
10523
10524
10527
10528
10530
10532
10533
10535
10536
10538
10541
10543
10547
10548
10549
10550
10552
10553
10560
10562
10566
10567
10570
10573
10576
10577
10579
10580
10583
10588
10589
10590
10591
10594
10595
10598
10601
10603
10604
10605
10606
10607
10701
10703
10704
10705
10706
10707
10708
10709
10710
10801
10803
10804
10805
10901
10913
10916
10917
10918
10919
10920
10921
10923
10924
10925
10926
10927
10928
10930
10940
10941
10950
10952
1095

In [36]:
pairs_copy = pd.DataFrame(pairs,columns=['l1','l2'])
pairs_copy.to_csv('pairs_save.csv')


### target function

In [37]:
expand_cost_expr = quicksum((20000+200*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step1']/fac[fac['facility_id'] == f]['total_capacity'].values[0]
                       +(20000+400*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step2']/fac[fac['facility_id'] == f]['total_capacity'].values[0]
                       +(20000+1000*fac[fac['facility_id'] == f]['total_capacity'].values[0])*z[f, 'step3']/fac[fac['facility_id'] == f]['total_capacity'].values[0] for f in exist_fac)
    
new_fac_cost_expr = quicksum(x[j, k] * new_fac_cost[new_fac_cost['size'] == k]['cost'].values[0] for j in loc_str_list for k in new_fac_size)

young_slots_expr = gp.quicksum(100 * y[f] for f in exist_fac)

m.setObjective(new_fac_cost_expr + expand_cost_expr + young_slots_expr, GRB.MINIMIZE)

### adding constrains

In [40]:
from gurobipy import quicksum, min_, Var, LinExpr


# (0) constraining on the dist<=0.06 pairs
for (l1,l2) in pairs:
    m.addConstr(
        quicksum(x[l1,size]+x[l2,size] for size in new_fac_size) <= 1, name = f"distance_restriction_{l1},{l2}"
    )

print('inter-newsteablishments distance limit done')

#(1) no desert
for code in zipcode:
    i_ = potential_loc[potential_loc["zip_code"]==code]["location_string"].to_list()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(z[f,s] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for s in steps) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == code]['is_high_demand'].values[0], f"no_desert_high_demand_{code}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(z[f,s] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for s in steps) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{code}")

print('1 done')

#(2) under5
for code in zipcode:
    i_ = potential_loc[potential_loc["zip_code"]==code]["location_string"].to_list()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == code]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == code]['-5'].values[0], f"under5_demand_{code}")

print('2 done')

#(3) expansion maximum
for f in exist_fac:
    m.addConstr(z[f, 'step1'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.1)
    m.addConstr(z[f, 'step2'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.05)
    m.addConstr(z[f, 'step3'] / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.05)
    if fac[fac['facility_id'] == f]['total_capacity'].values[0] < 500:
        m.addConstr(quicksum(z[f, s] for s in steps) <= 500-fac[fac['facility_id'] == f]['total_capacity'].values[0])
    else:
        m.addConstr(quicksum(z[f, s] for s in steps) == 0)
    m.addConstr(y[f] <= quicksum(z[f, s] for s in steps))
    m.addConstr(y[f] >= 0)
    for s in steps:
        m.addConstr(z[f, s] >= 0)

for i in potential_loc["location_string"].to_list():
    m.addConstr(quicksum(x[i, j] for j in new_fac_size) <= 1, f'new_facility_num_{i}')

print('3 done')
# constraining on the locations: for all news built facilities, distance to the nearest existing facility must be greater than 0.06miles


#for code in zipcode:
#    loc_in_this_zip = potential_loc[potential_loc["zip_code"]==code]['location_string'].tolist()
#    # this is a list of 
#    #x_sum = {l: quicksum(x[l, j] for j in new_fac_size) for l in loc_in_this_zip}
#    for l1 in loc_in_this_zip:
#        for l2 in loc_in_this_zip:
#            if l1 < l2:
#                dist = loc_distance(float(l1.split(",")[0]),float(l1.split(",")[1]), float(l2.split(",")[0]),float(l2.split(",")[1]))
#                m.addConstr(
#                    dist >= 0.06 #+ (1 - x_sum[l1]) * 100000 + (1 - x_sum[l2]) * 100000,
#                    ,name=f'distance_{l1}_{l2}'
#                )

#print('4 done')
# defining the amount of facilities built within each steps using constraint
#for f in exist_fac:
#    m.addConstr(tf1[f] == y[f, 'under5'] + y[f, 'over5'])
#    m.addConstr(tf2[f] == y[f, 'under5'] + y[f, 'over5'] - 0.1 * fac[fac['facility_id'] == f]['total_capacity'].values[0])
#    m.addConstr(tf3[f] == y[f, 'under5'] + y[f, 'over5'] - 0.15 * fac[fac['facility_id'] == f]['total_capacity'].values[0])
#for f in exist_fac:
#    m.addConstr(c1[f] == min_(tf1[f], constant = 0.1 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))
#    m.addConstr(c2[f] == min_(tf2[f], constant =  0.15 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))
#    m.addConstr(c3[f] == min_(tf3[f], constant =  0.05 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))

m.update()

#(5) constraint for var z
#for in exist_fac:
#    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.1 + (1 - z[f, 'step1']) * 1000, 'step1_limit')
#    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] >= 0.1 + z[f, 'step2'], 'step2_limit')
#    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.15 + (1 - z[f, 'step2']) * 1000, 'step2_upper_limit')
#    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] >= 0.15 + z[f, 'step3'], 'step3_limit')
#    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.2 + (1 - z[f, 'step3']) * 1000, 'step3_upper_limit')


inter-newsteablishments distance limit done
1 done
2 done
3 done


In [41]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 553574 rows, 365920 columns and 3533052 nonzeros
Model fingerprint: 0x2b2b3366
Variable types: 0 continuous, 365920 integer (306900 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 8e+03]
Presolve removed 482238 rows and 113117 columns
Presolve time: 0.76s
Presolved: 71336 rows, 252803 columns, 592760 nonzeros
Variable types: 0 continuous, 252803 integer (238582 binary)
Found heuristic solution: objective 3.604453e+08
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5944882e+08   2.778673e+01   2.920017e+07      8s
      13    1